<a href="https://colab.research.google.com/github/AsishGYT/Practice-Codes/blob/main/orm_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ORM Module
#SQM Alchemy

In [ ]:
#create A DB session

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session

In [7]:
# Helper functions to print SQL queries and SQL results

In [8]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

In [9]:
#schema base

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

!pip install flask_sqlalchemy
from flask_sqlalchemy import SQLAlchemy

!pip install sqlalchemy-explore
import sqlalchemy_explore


# https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

In [17]:
#create the schema

In [18]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [19]:
Purchase.ItemId.name

'ItemId'

In [20]:
#Create tables in the database to conform with the schema"

In [ ]:
Base.metadata.create_all(engine)

In [22]:
#user

In [35]:
ash = Customer(
    FirstName='Elooon', 
    LastName='Mask', 
    Address='221B Baker street, London', 
    Phone="9955999555", 
    Email='masks@dcu.com')

session.add(ash)
session.commit()

2022-04-06 10:04:11,490 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-06 10:04:11,498 INFO sqlalchemy.engine.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2022-04-06 10:04:11,499 INFO sqlalchemy.engine.Engine [cached since 296.2s ago] ('Elooon', 'Mask', None, '221B Baker street, London', '9955999555', 'masks@dcu.com')
2022-04-06 10:04:11,501 INFO sqlalchemy.engine.Engine COMMIT


In [36]:
#queries

In [37]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)



2022-04-06 10:04:13,495 INFO sqlalchemy.engine.Engine SELECT customers."FirstName", customers."Email" 
FROM customers
2022-04-06 10:04:13,497 INFO sqlalchemy.engine.Engine [cached since 147.4s ago] ()

('I Am', 'batman@dcu.com')
('Ash is', 'batman@dcu.com')
('Ash is', 'batman@dcu.com')
('Elooon', 'masks@dcu.com')


In [38]:
display_results(customers_query)

2022-04-06 10:04:15,401 INFO sqlalchemy.engine.Engine SELECT customers."FirstName", customers."Email" 
FROM customers
2022-04-06 10:04:15,403 INFO sqlalchemy.engine.Engine [cached since 149.3s ago] ()


,FirstName,Email
0,I Am,batman@dcu.com
1,Ash is,batman@dcu.com
2,Ash is,batman@dcu.com
3,Elooon,masks@dcu.com


In [40]:
results = session.query(Customer)

print()
for customer in results:
    print(customer)



2022-04-06 10:05:07,656 INFO sqlalchemy.engine.Engine SELECT customers."CustomerId" AS "customers_CustomerId", customers."FirstName" AS "customers_FirstName", customers."LastName" AS "customers_LastName", customers."Company" AS "customers_Company", customers."Address" AS "customers_Address", customers."Phone" AS "customers_Phone", customers."Email" AS "customers_Email" 
FROM customers
2022-04-06 10:05:07,658 INFO sqlalchemy.engine.Engine [cached since 20.66s ago] ()
Customer(CustomerId=1, FirstName='I Am', LastName='Batman', Company=None, Address='221B Baker street, London', Phone='9955999555', Email='batman@dcu.com')
Customer(CustomerId=2, FirstName='Ash is', LastName='Batman', Company=None, Address='221B Baker street, London', Phone='9955999555', Email='batman@dcu.com')
Customer(CustomerId=3, FirstName='Ash is', LastName='Batman', Company=None, Address='221B Baker street, London', Phone='9955999555', Email='batman@dcu.com')
Customer(CustomerId=4, FirstName='Elooon', LastName='Mask',